<a href="https://colab.research.google.com/github/ozcannrabia/feature-engineering/blob/main/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ham veriden değişken türetmek

Binary Features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_curve
from xgboost import XGBClassifier

In [ ]:
def load():
  df = pd.read_csv("titanic.csv")
  return df
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype('int')
df.head()
# cabin kolonunda boş olanlara 0, dolu olanlara 1 değerini ver

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [ ]:
df.groupby("NEW_CABIN_BOOL").agg({"Survived":"mean"})

,Survived
NEW_CABIN_BOOL,
0,0.299854
1,0.666667


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),
                                             df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],

                                      nobs=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],
                                            df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat = 9.4597, p-value = 0.0000


In [ ]:
# cabin sınıfı 1 veya 0 olup hayatta kalıp kalmama durumu arasında fark yoktur
# yukarıdaki hipotez p-value 0 çıktığı için reddedilir yani anlamlı bir farklılık vardır

In [ ]:
df.loc[((df["SibSp"] + df["Parch"]) > 0), "NEW_IS_ALONE"] = "NO"
df.loc[((df["SibSp"] + df["Parch"]) == 0), "NEW_IS_ALONE"] = "YES"
# akrabası olup olmama durumu targetta fark yaratabilir

In [ ]:
df.groupby("NEW_IS_ALONE").agg({"Survived":"mean"})

,Survived
NEW_IS_ALONE,
NO,0.505650
YES,0.303538


In [ ]:
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].sum(),
                                             df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].sum()],

                                      nobs=[df.loc[df["NEW_IS_ALONE"] == "YES", "Survived"].shape[0],
                                            df.loc[df["NEW_IS_ALONE"] == "NO", "Survived"].shape[0]])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -6.0704, p-value = 0.0000


Text Features

In [ ]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Letter Count

In [ ]:
df["NEW_NAME_COUNT"] = df["Name"].str.len()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24


Word Count

In [ ]:
df["NEW_NAME_WORD_COUNT"] = df["Name"].apply(lambda x: len(str(x).split(" ")))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_NAME_COUNT,NEW_NAME_WORD_COUNT
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,23,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,22,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,7
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,24,4


Özel Yapıları Yakalamak

In [ ]:
df["NEW_NAME_DR"] = df["Name"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))
df.groupby("NEW_NAME_DR").agg({"Survived":["mean", "count"]})

Survived      
                mean count
NEW_NAME_DR               
0            0.38252   881
1            0.50000    10

Regex Features

In [ ]:
df["NEW_TITLE"] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df[["NEW_TITLE", "Survived", "Age"]].groupby(["NEW_TITLE"]).agg({"Survived":"mean", "Age":["count","mean"]})

Survived   Age           
               mean count       mean
NEW_TITLE                           
Capt       0.000000     1  70.000000
Col        0.500000     2  58.000000
Countess   1.000000     1  33.000000
Don        0.000000     1  40.000000
Dr         0.428571     6  42.000000
Jonkheer   0.000000     1  38.000000
Lady       1.000000     1  48.000000
Major      0.500000     2  48.500000
Master     0.575000    36   4.574167
Miss       0.697802   146  21.773973
Mlle       1.000000     2  24.000000
Mme        1.000000     1  24.000000
Mr         0.156673   398  32.368090
Mrs        0.792000   108  35.898148
Ms         1.000000     1  28.000000
Rev        0.000000     6  43.166667
Sir        1.000000     1  49.000000

In [ ]:
# her title a göre eksik yaşları bunların ortalamasına göre doldurursak daha iyi olur

Date Features

In [ ]:
def load_insta():
  df = pd.read_csv("Instagram_Data.csv")
  return df
df = load_insta()

In [ ]:
df["Date Posted"] = pd.to_datetime(df["Date Posted"], format="%Y-%m-%d %H:%M:%S")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178922 entries, 0 to 178921
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   User uuid              178922 non-null  int64         
 1   Likes                  178922 non-null  int64         
 2   Days passed from post  178922 non-null  int64         
 3   Likes Score            178922 non-null  float64       
 4   Type                   178922 non-null  object        
 5   Numer of Tags          178922 non-null  int64         
 6   Numer of Comments      178922 non-null  int64         
 7   Date Posted            178922 non-null  datetime64[ns]
 8   Year                   178922 non-null  int64         
 9   Month                  178922 non-null  int64         
 10  Day                    178922 non-null  int64         
 11  Hour                   178922 non-null  int64         
 12  Minute                 178922 non-null  int6

In [ ]:
df["year"] = df["Date Posted"].dt.year
df["month"] = df["Date Posted"].dt.month
df["day"] = df["Date Posted"].dt.day
df.head()

,User uuid,Likes,Days passed from post,Likes Score,Type,Numer of Tags,Numer of Comments,Date Posted,Year,Month,Day,Hour,Minute,year,month,day
0,1,506,657,0.74,Image,3,11,2019-05-12 21:18:39,2019,5,12,21,18,2019,5,12
1,1,385,657,0.57,Image,0,1,2019-05-12 14:25:55,2019,5,12,14,25,2019,5,12
2,1,373,666,0.55,Images,4,15,2019-05-03 15:18:37,2019,5,3,15,18,2019,5,3
3,1,312,667,0.46,Images,4,1,2019-05-02 16:40:38,2019,5,2,16,40,2019,5,2
4,1,513,674,0.75,Images,5,0,2019-04-25 16:51:58,2019,4,25,16,51,2019,4,25


Feature Interactions -> Özellik Etkileşimleri

In [ ]:
df = load()
df["NEW_AGE_PCLASS"] = df["Age"] * df["Pclass"]
df["NEW_FAMILY_SIZE"] = df["SibSp"] + df["Parch"] + 1

In [ ]:
# Tüm değerleri küçük harfe dönüştür
df['Sex'] = df['Sex'].str.lower()

# NEW_SEX_CAT sütununu sıfırla
df['NEW_SEX_CAT'] = None

# Kadınlar için
df.loc[(df['Sex'] == 'female') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['Sex'] == 'female') & (df['Age'] > 21) & (df['Age'] <= 50), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['Sex'] == 'female') & (df['Age'] > 50), 'NEW_SEX_CAT'] = 'seniorfemale'

# Erkekler için
df.loc[(df['Sex'] == 'male') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['Sex'] == 'male') & (df['Age'] > 21) & (df['Age'] <= 50), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['Sex'] == 'male') & (df['Age'] > 50), 'NEW_SEX_CAT'] = 'seniormale'

# Kontrol için sonuçları yazdır
print(df['NEW_SEX_CAT'].value_counts())


NEW_SEX_CAT
maturemale      286
maturefemale    160
youngmale       120
youngfemale      84
seniormale       47
seniorfemale     17
Name: count, dtype: int64


In [ ]:
df.groupby("NEW_SEX_CAT")["Survived"].mean()

,Survived
NEW_SEX_CAT,
maturefemale,0.775000
maturemale,0.199301
seniorfemale,0.941176
seniormale,0.127660
youngfemale,0.678571
youngmale,0.250000


In [ ]:
# yaşları ve cinsiyetleri kategorileştirip önemine baktık ve bizim için iyi bir yeni kolon oluşturduğunu gördük